<h3>7. Algorithm Design </h3>

A major part of algorithmic problem solving is selecting or adapting an appropriate algorithm for the problem at hand. The best known strategy is known as divide-and-conquer. We attack a problem of size n by dividing it into two problems of size n/2, solve these problems, and combine their results into a solution of the original problem. 

**Divide-and-Conquer**

Sorting by Divide-and-Conquer: To sort an array, we split it in half and sort each half (recursively); we merge each sorted half back into a whole list (again recursively); this algorithm is known as "Merge Sort".

![alt text](https://scontent.fist2-4.fna.fbcdn.net/v/t1.15752-9/64789182_2601548716523414_7838641960347762688_n.png?_nc_cat=111&_nc_ht=scontent.fist2-4.fna&oh=816cf0bb633ee359979ea0271ed390b2&oe=5D8F60B7)

**Binary Search**

Another example is the process of looking up a word in a dictionary. We open the book somewhere around the middle and compare our word with the current page. If it's earlier in the dictionary we repeat the process on the first half; if its later we use the second half. This search method is called binary search since it splits the problem in half at every step.

**Recursion**

We define a function f which simplifies the problem, and calls itself to solve one or more easier instances of the same problem. It then combines the results into a solution for the original problem.

**Example**

For example, suppose we have a set of n words, and want to calculate how many different ways they can be combined to make a sequence of words. If we have only one word (n=1), there is just one way to make it into a sequence. If we have a set of two words, there are two ways to put them into a sequence. For three words there are six possibilities. In general, for n words, there are n × n-1 × … × 2 × 1 ways (i.e. the factorial of n). We can code this up as follows:

In [0]:
def factorial1(n):
     result = 1
     for i in range(n):
      result *= (i+1)
      return result

Suppose we have a way to construct all orderings for n-1 distinct words. Then for each such ordering, there are n places where we can insert a new word: at the start, the end, or any of the n-2 boundaries between the words. Thus we simply multiply the number of solutions found for n-1 by the value of n. We also need the base case, to say that if we have a single word, there's just one ordering. We can code this up as follows:

In [0]:
def factorial2(n):
     if n == 1:
         return 1
    else:
         return n * factorial2(n-1)

**Space-Time Tradeoffs**

We can sometimes significantly speed up the execution of a program by building an auxiliary data structure, such as an index. The following example implements a simple text retrieval system for the Movie Reviews Corpus. By indexing the document collection it provides much faster lookup.

In [0]:
def raw(file):
    contents = open(file).read()
    contents = re.sub(r'<.*?>', ' ', contents)
    contents = re.sub('\s+', ' ', contents)
    return contents

def snippet(doc, term):
    text = ' '*30 + raw(doc) + ' '*30
    pos = text.index(term)
    return text[pos-30:pos+30]

print("Building Index...")
files = nltk.corpus.movie_reviews.abspaths()
idx = nltk.Index((w, f) for f in files for w in raw(f).split())

query = ''
while query != "quit":
    query = input("query> ")     # use raw_input() in Python 2
    if query in idx:
        for doc in idx[query]:
            print(snippet(doc, query))
    else:
        print("Not found")

**Vocabulary List**

Maintaing a vocabulary list is an example of space-time tradeoff, if you need to process an input text to check that all words are in an existing vocabulary, the vocabulary should be stored as a set, not a list. The elements of a set are automatically indexed, so testing membership of a large set will be much faster than testing membership of the corresponding list.



In [0]:
from timeit import Timer
vocab_size = 100000
setup_list = "import random; vocab = range(%d)" % vocab_size 
#simulates a vocabulary of 100,000 items using a list
setup_set = "import random; vocab = set(range(%d))" % vocab_size
#or set of integers
statement = "random.randint(0, %d) in vocab" % (vocab_size * 2)    
#will generate a random item which has a 50% chance of being in the vocabulary
print(Timer(statement, setup_list).timeit(1000))
print(Timer(statement, setup_set).timeit(1000))

**Result** Performing 1000 list membership tests takes a total of 2.8 seconds, while the equivalent tests on a set take a mere 0.0037 seconds, or three orders of magnitude faster.

**Dynamic Programming**

Dynamic programming is a particular method used in natural language processing to design algorithms. The word ' programming ' is used to mean planning or scheduling in a distinct context than you might expect. In the remainder of this section we will introduce dynamic programming, but in a rather different context to syntactic parsing.

For example, Pingala was an Indian author who lived around the 5th century B.C., and wrote a treatise on Sanskrit prosody called the Chandas Shastra. Virahanka extended this work around the 6th century A.D., studying the number of ways of combining short and long syllables to create a meter of length n. Short syllables, marked S, take up one unit of length, while long syllables, marked L, take two. Pingala found, for example, that there are five ways to construct a meter of length 4: V4 = {LL, SSL, SLS, LSS, SSSS}. Observe that we can split V4 into two subsets, those starting with L and those starting with S.

V4 =
  LL, LSS
    i.e. L prefixed to each item of V2 = {L, SS}
  SSL, SLS, SSSS
    i.e. S prefixed to each item of V3 = {SL, LS, SSS}

In [0]:
def virahanka1(n):  #to compute meters
    if n == 0:
        return [""]
    elif n == 1:
        return ["S"]
    else:
        s = ["S" + prosody for prosody in virahanka1(n-1)]
        l = ["L" + prosody for prosody in virahanka1(n-2)]
        return s + l

def virahanka2(n):
    lookup = [[""], ["S"]]
    for i in range(n-1):
        s = ["S" + prosody for prosody in lookup[i+1]]
        l = ["L" + prosody for prosody in lookup[i]]
        lookup.append(s + l)
    return lookup[n]

def virahanka3(n, lookup={0:[""], 1:["S"]}):
    if n not in lookup:
        s = ["S" + prosody for prosody in virahanka3(n-1)]
        l = ["L" + prosody for prosody in virahanka3(n-2)]
        lookup[n] = s + l
    return lookup[n]

from nltk import memoize  #stores the result of each previous call to the function along with the parameters that were used
@memoize
def virahanka4(n):
    if n == 0:
        return [""]
    elif n == 1:
        return ["S"]
    else:
        s = ["S" + prosody for prosody in virahanka4(n-1)]
        l = ["L" + prosody for prosody in virahanka4(n-2)]
        return s + l

These are the Four Ways to Compute Sanskrit Meter: (i) recursive; (ii) bottom-up dynamic programming; (iii) top-down dynamic programming; and (iv) built-in memoization.

![alt text](https://scontent.fist2-3.fna.fbcdn.net/v/t1.15752-9/64923758_696010734183994_3334067401170878464_n.png?_nc_cat=106&_nc_ht=scontent.fist2-3.fna&oh=98032284ef25bdf364474a38d9be57e0&oe=5D8F6044)

**8. A Sample of Python Libraries**

**Matplotlib**

Python has some libraries that are useful for visualizing language data. The Matplotlib package supports sophisticated plotting functions with a MATLAB-style interface, and is available from http://matplotlib.sourceforge.net/.


So far we have focused on textual presentation and the use of formatted print statements to get output lined up in columns. It is often very useful to display numerical data in graphical form, since this often makes it easier to detect patterns.

**Frequency of Modals in Different Sections of the Brown Corpus**

In [0]:
from numpy import arange
from matplotlib import pyplot

colors = 'rgbcmyk' # red, green, blue, cyan, magenta, yellow, black

def bar_chart(categories, words, counts):
    "Plot a bar chart showing counts for each word by category"
    ind = arange(len(words))
    width = 1 / (len(categories) + 1)
    bar_groups = []
    for c in range(len(categories)):
        bars = pyplot.bar(ind+c*width, counts[categories[c]], width,
                         color=colors[c % len(colors)])
        bar_groups.append(bars)
    pyplot.xticks(ind+width, words)
    pyplot.legend([b[0] for b in bar_groups], categories, loc='upper left')
    pyplot.ylabel('Frequency')
    pyplot.title('Frequency of Six Modal Verbs by Genre')
    pyplot.show()

![alt text](https://scontent.fist2-4.fna.fbcdn.net/v/t1.15752-9/65201016_615619188916293_7510837201479925760_n.png?_nc_cat=105&_nc_ht=scontent.fist2-4.fna&oh=5425428a9de35900134908b96799e0cc&oe=5DC39987)

ar Chart Showing Frequency of Modals in Different Sections of Brown Corpus: this visualization was produced by the program. From the bar chart it is immediately obvious that may and must have almost identical relative frequencies. The same goes for could and might.

**NetworkX** 

The NetworkX package is for defining and manipulating structures consisting of nodes and edges, known as graphs. It is available from https://networkx.lanl.gov/. NetworkX can be used in conjunction with Matplotlib to visualize networks, such as WordNet (the semantic network)

**Using the NetworkX and Matplotlib Libraries**

In [0]:
import networkx as nx
import matplotlib
from nltk.corpus import wordnet as wn

def traverse(graph, start, node):
    graph.depth[node.name] = node.shortest_path_distance(start)
    for child in node.hyponyms():
        graph.add_edge(node.name, child.name) 
        traverse(graph, start, child) 
def hyponym_graph(start):
    G = nx.Graph() 
    G.depth = {}
    traverse(G, start, start)
    return G

def graph_draw(graph):
    nx.draw_graphviz(graph,
         node_size = [16 * graph.degree(n) for n in graph],
         node_color = [graph.depth[n] for n in graph],
         with_labels = False)
    matplotlib.pyplot.show()

![alt text](https://scontent.fist2-4.fna.fbcdn.net/v/t1.15752-9/65431496_2324504134482507_678848602226819072_n.png?_nc_cat=111&_nc_ht=scontent.fist2-4.fna&oh=c7e36c359596667a4af0ae77409b82e2&oe=5D7EE329)

Visualization with NetworkX and Matplotlib: Part of the WordNet hypernym hierarchy is displayed, starting with dog.n.01 (the darkest node in the middle); node size is based on the number of children of the node, and color is based on the distance of the node from dog.n.01; this visualization was produced by the program.


**csv**

Language analysis work often involves data tabulations, containing information about lexical items, or the participants in an empirical study, or the linguistic features extracted from a corpus. Here's a fragment of a simple lexicon, in CSV format:

sleep, sli:p, v.i, a condition of body and mind ...
walk, wo:k, v.intr, progress by lifting and setting down each foot ...
wake, weik, intrans, cease to sleep

In [0]:
import csv
 input_file = open("lexicon.csv", "rb")  #to open a CSV file called lexicon.csv
 for row in csv.reader(input_file):   #iterate over rows
     print(row)

**NumPy**

The NumPy package provides substantial support for numerical processing in Python. NumPy has a multi-dimensional array object, which is easy to initialize and access:

In [0]:
from numpy import array
 cube = array([ [[0,0,0], [1,1,1], [2,2,2]],
                [[3,3,3], [4,4,4], [5,5,5]],
                [[6,6,6], [7,7,7], [8,8,8]] ])
 cube[1,1,1]
 cube[2].transpose()
 cube[2,1:]

NumPy includes linear algebra functions. Here we perform singular value decomposition on a matrix, an operation used in latent semantic analysis to help identify implicit concepts in a document collection.

In [0]:
from numpy import linalg
 a=array([[4,0], [3,-5]])
u,s,vt = linalg.svd(a)
u
s
vt

**Other Python Libraries**

There are many other Python libraries, and you can search for them with the help of the Python Package Index http://pypi.python.org/. Many libraries provide an interface to external software, such as relational databases (e.g. mysql-python) and large document collections (e.g. PyLucene). Many other libraries give access to file formats such as PDF, MSWord, and XML (pypdf, pywin32, xml.etree), RSS feeds (e.g. feedparser), and electronic mail (e.g. imaplib, email).